In [1]:
from FNN import FNN
from Layer import Layer
from Neuron import Neuron
import numpy as np
from ActivFunctions import  *
from InitFunctions import  *
from SuppFunctions import  *
from ErrorClasses import *

Neuron Class functionality testing

In [2]:
#zero initialization
neuron = Neuron(2,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [3]:
#random initialization
neuron = Neuron(2,identity,method_ini = "RandomNor", random_mean = -1000.0, random_std = 10.0)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [4]:
#initialization from list
vector = [1.0,2.0,3.5]
neuron = Neuron(vector,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [5]:
#initialization from vector
vector = np.array([1.0,2.0,3.5])
neuron = Neuron(vector,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [6]:
#initialization from row vector(2D array)
vector = np.array([1.0,2.0,3.5])
vector_row = vector[np.newaxis,:]
neuron = Neuron(vector_row,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [7]:
#initialization from column vector(2D array)
vector = np.array([1.0,2.0,3.5])
vector_column = vector[:,np.newaxis]
neuron = Neuron(vector_column,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [8]:
#error messages testing
print("Test for: no function given as activation function")
try:
    vector = np.array([1.0,2.0,3.5])
    errorNeuron = Neuron(vector,1)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (2 dim)")
try:
    vector = np.array([[1.0,2.0,3.5],[1.0,2.0,3.5]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (higher than 2 dim)")
try:
    vector = np.array([[[1.0,2.0,3.5],[1.0,2.0,3.5]],[[1.0,2.0,3.5],[1.0,2.0,3.5]]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong weights given for initialization")
try:
    vector = np.array(["1.0","2.0","3.5"])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given in list for weight initialization")
try:
    vector = ["as","2.0","3.5"]
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - inproper dimension of input")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = np.array([[[1.0,1.0],[1.0,1.0]],[[1.0,1.0],[1.0,1.0]]]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - wrong value types in input: they are not rational numbers")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = np.array([1.0 + 3j,1.0]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type for list")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = ["a","1.0"]
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = "a"
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too small)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorSmall = np.array([1.0])
    neuronTest.forward(inputTestErrorSmall)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too big)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorBig = np.array([[1.0],[1.0],[1.0],[1.0]])
    neuronTest.forward(inputTestErrorBig)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: no function given as activation function
Correct errors was caught. Error: Given input is not supported by implementation for activation function initialization. Given variable is not a function and thus can not be used as activation function.
Test for: not vector given for weights initialization (2 dim)
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given variable is not an vector (not column or row vector in 2 dim), so it is unsuitable for neuron weight initialization.
Test for: not vector given for weights initialization (higher than 2 dim)
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given variable is not an vector (due to more than 2 dim), so it is unsuitable for neuron weight initialization.
Test for: wrong weights given for initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization.

Layer class implementation testing

In [9]:
#initialization by number of dim in vector
dim_layer = np.array([4,5])
testLayer = Layer(dim_layer,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [10]:
#initialization by number of dim in list
dim_layer = [4,5]
testLayer = Layer(dim_layer,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [11]:
#initialization by giving weight array
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
testLayer = Layer(weights,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [12]:
#initialization by giving weight array and function (single) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
testLayer = Layer(weights,[identity],method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [13]:
#initialization by giving weight array and identity function (mutliple same) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
activation_functions = [identity,identity,identity]
testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [14]:
#initialization by giving weight array and identity function (mutliple different) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
activation_functions = [sigmoid,relu,leaky_relu]
testLayer = Layer(weights,activation_functions)
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [15]:
#error messages testing
print("Test for: wrong var type (not number) given in list for weight initialization")
try:
    vector = ["as","2.0"]
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given (number, but no int) in list for weight initialization")
try:
    vector = [1,2.3]
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type (not number) given in 1D array for weight initialization")
try:
    vector = np.array(["as","2.0"])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given (number, but no int) in 1D array for weight initialization")
try:
    vector = np.array([1,2.3])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong dimensions of given 1D array for weight initialization")
try:
    vector = np.array([1,2,1,1,2])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not rational numbers given in 2D array for weight initialization")
try:
    weights = np.array([[1.0 + 4j,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not enough information given in 2D array for weight initialization")
try:
    weights = np.array([[]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: bigger than 2D was given for array for weight initialization")
try:
    weights = np.array([[[2]]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not supported data type given for array for weight initialization")
try:
    weights = "hehe"
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not supported data type given for array for weight initialization")
try:
    weights = "hehe"
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not compatible amount of activation functions given to the amount of neurons for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,relu,leaky_relu,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not activation function given (inside list) for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,1,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not activation function given (standalone) for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = 1
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, too much dimensions in input")
try:
    #Input data for input propagation (forward pass)
    inputTestArray = np.array([[[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]],[[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestArray)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, complex input")
try:
    #Input data for input propagation (forward pass)
    inputTestVect = np.array([[1.0 + 3j],[1.0],[1.0],[1.0],[1.0]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestVect)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, wrong data type of input")
try:
    #Input data for input propagation (forward pass)
    inputTestVect = "sda"
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestVect)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, wrong data type of input in list")
try:
    #Input data for input propagation (forward pass)
    inputTestList =["a",1.0,1.0,1.0,1.0]
    inputTestArray = np.array([[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestList)
    testLayer.forward(inputTestArray)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, input does not match layer neurons")
try:
    #Input data for input propagation (forward pass)
    inputTestList = [1.0,1.0,1.0,1.0,1.0,1.0,1.0]
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestList)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: wrong var type (not number) given in list for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type given (number, but no int) in list for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type (not number) given in 1D array for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type given (number, but no int) in 1D array for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weight

FNN class implementation testing

In [16]:
#initialization by number of dim in vector(list)
dim_layer = [5,5,3,2,1]
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
print(testNet.weights_list[0])
print(testNet.weights_list[1])
print(testNet.weights_list[2])
print(testNet.weights_list[3])
print(testNet.activ_functions_list_list[0])
print(testNet.activ_functions_list_list[1])
print(testNet.activ_functions_list_list[2])
print(testNet.activ_functions_list_list[3])
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

[[-1.1179869  -1.08994068 -1.00314658  0.40362996  0.40249222  1.53077109]
 [-1.12299715  0.88838418  1.17020152  0.03930541 -1.05886099  1.27695219]
 [ 0.66879362 -0.382688    1.65803616  1.11591284  0.25031781  1.53800258]
 [-0.13890703 -1.03001466 -1.91954944 -0.71672363  1.50396064 -0.77648341]
 [ 0.75851061 -0.4936532   0.32591992 -0.1412931   0.86947368  0.3799247 ]]
[[ 0.40729626 -0.23484442 -0.92713911 -0.10323146  0.26776721 -1.09578581]
 [-0.52312979  0.28325855 -0.41142413  1.23140183  0.90936315 -0.39611024]
 [-0.5498238   1.33807313  0.00868692 -1.07741143  0.9717507   0.39137083]]
[[-0.20290865  1.52390804 -1.64769309 -1.18657586]
 [ 0.83594966  0.02997074  0.60967493 -0.02472798]]
[[-1.18407896  0.49134787 -0.65479313]]
[<function relu at 0x000002CB6A13F420>, <function relu at 0x000002CB6A13F420>, <function relu at 0x000002CB6A13F420>, <function relu at 0x000002CB6A13F420>, <function relu at 0x000002CB6A13F420>]
[<function relu at 0x000002CB6A13F420>, <function relu at 0

True

In [17]:
#initialization by number of dim in vector(1D array)
dim_layer = np.array([5,5,3,2,1])
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [18]:
#dummy network ini
dim_layer = np.array([5,5,3,2,1])
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#initialization by ready weight array
dim_layer = testNet.weights_list
print(dim_layer)
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

[array([[ 6.57048552e-01,  5.98076423e-01,  1.91901816e-01,
        -6.94000471e-01, -5.63313279e-04,  1.26024002e+00],
       [-7.46135159e-01, -8.84873052e-01,  2.53064619e+00,
         5.26528566e-01,  1.61440789e-01, -2.66098102e-01],
       [ 1.01072397e+00, -7.63319553e-01,  9.56418671e-02,
         5.13133161e-01,  1.20738738e+00,  1.48489623e+00],
       [-5.62912841e-01,  1.22182365e-01, -1.75497643e+00,
         4.49763062e-01,  1.05281982e+00, -3.02800771e-01],
       [-8.45365687e-01, -1.41479892e+00, -3.72318981e-01,
        -1.78216489e+00,  7.75335461e-01,  3.04262331e-01]]), array([[-1.42102294,  2.11267689, -0.15460401, -0.70140249,  0.37909692,
        -0.09411567],
       [ 0.15248484,  0.24628693, -1.11929866,  1.55942963, -0.73391838,
         1.03354076],
       [ 0.49128129, -1.48642351, -0.23805543,  3.09905986, -1.26822671,
         1.11913712]]), array([[-1.46807517,  1.47902898, -0.06590619,  0.05498133],
       [-1.08931023, -0.4263861 ,  0.75493781,  0.3531

True

In [19]:
#initialization by number of dim in vector(1D array) and 1 activation function(as value)
dim_layer = np.array([5,5,3,2,1])
activ_func = identity
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [20]:
#initialization by number of dim in vector(1D array) and 1 activation function(as list)
dim_layer = np.array([5,5,3,2,1])
activ_func = [identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [21]:
#initialization by number of dim in vector(1D array) and 2 activation functions
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [22]:
#initialization by number of dim in vector(1D array) and activation function for each layer
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,sigmoid,leaky_relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [23]:
#initialization by number of dim in vector(1D array) and activation function for each neuron
dim_layer = np.array([5,5,3,2,1])
l1 = [relu,sigmoid,sigmoid,leaky_relu,identity]
l2 = [relu,sigmoid,sigmoid]
l3 = [relu,sigmoid]
l4 = [identity]
activ_func =  [l1,l2,l3,l4]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [24]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Zero weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="Zero")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [25]:
#initialization by number of dim in vector(1D array) and 2 activation functions with random uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [26]:
#initialization by number of dim in vector(1D array) and 2 activation functions with random normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [27]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Xavier uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="XavUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [28]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Xavier normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="XavNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [29]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [30]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [31]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeNor")
#
layers_list = testNet.decomposeIntoLayers()
neurons_list_list = testNet.decomposeIntoNeurons()
#
print(layers_list)
print(neurons_list_list)

NotSupportedInputGiven: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.

In [ ]:
#error messages testing
print("Test for: not supported input for weights")
try:
    dim_layer = "[5,5,3,2,1]"
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: array of not supported dimension")
try:
    dim_layer = np.array([[5,5,3,2,1],[5,5,3,2,1]])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: given numbers are not integers")
try:
    dim_layer = np.array([5,5,3.1,2,1])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not enough numbers given. Minimally required input and output layers cannot be created")
try:
    dim_layer = np.array([5])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Reload Modules


In [ ]:
import importlib
import sys
import numpy as np

# Reload order:
modules_in_order = [
    "ActivFunctions",
    "SuppFunctions",
    "InitFunctions",
    "Layer",
    "FNN",
    "TrainingFunctions"
]

for m in modules_in_order:
    if m in sys.modules:
        importlib.reload(sys.modules[m])
    else:
        globals()[m] = importlib.import_module(m)

print("All modules reloaded successfully.")


Imports

In [ ]:
from FNN import FNN
from TrainingFunctions import backwards
from LossFunctions import MeanSquaredErrorDerivative

# Activations
from ActivFunctions import identity, sigmoid, tanh, relu, leaky_relu
import numpy as np


In [ ]:
print("=== Test 1: Single-Layer Network Gradient ===")

dim_layer = np.array([4, 5])   # 4 inputs -> 5 outputs
testNet = FNN(dim_layer, identity, method_ini="RandomNor")

inputTestVect = np.ones((4, 1))
targetVect = 0.5 * np.ones((5, 1))

grads = backwards(testNet, inputTestVect, targetVect, MeanSquaredErrorDerivative)

print("\nGradient matrix for single layer:")
print(grads[0])


In [ ]:
print("=== Test 2: Deep 10-Layer Network ===")

dim_layer = np.array([
    5, 8, 6, 7, 5, 9, 4, 3, 2, 4, 1
])

deepNet = FNN(dim_layer, identity, method_ini="RandomNor")

inputVect = np.ones((5, 1))
targetVect = np.array([[0.7]])

grads = backwards(deepNet, inputVect, targetVect, MeanSquaredErrorDerivative)

for i, g in enumerate(grads):
    print(f"\nGradient matrix for layer {i}:")
    print(g)
